<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EA%B5%90%EA%B3%BC%EC%84%B8%ED%8A%B9_%EC%A0%84%EC%B2%98%EB%A6%ACRPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 교과세특 전처리 과정
###1. 불필요한 1~3행을 삭제
###2. '번 호'를 번호로 수정
###3. '성 명'을 성명으로 수정
###3. '학 년'을 학년으로 수정
###4. '세부능력 및 특기사항'을 '세특'으로 수정
###5. 인덱스(10번째 행)을 index(첫번째 열)로 이동시키기
###5. 열 '성명' 다음행의 '성명'이 비어 있고, '세특' 내용이 빈칸에 아니면(있으면) 바로 앞 행의 성명으로 대체
###6. '성명' 열이 빈칸이 아닌데 번호가 비어

In [2]:
#세특 내용을 합치기~
import pandas as pd
import openpyxl


# Read the Excel file into a pandas DataFrame
path = '/content/drive/MyDrive/python/deungchon/'
df = pd.read_csv(f'{path}srecord_306_raw_full.csv', encoding='cp949')


#1. 불필요한 행삭제: 1~3행 삭제
df = df.drop(range(0, 6))


#2. 이름과 세특 열 사이의 불필요한 'Unnamed: 3' column 삭제
df = df.drop(labels='Unnamed: 3', axis=1)


#3. remove rows with only NaN values
df = df.dropna(how='all')

#4. 비어있는 5번째열부터 끝까지 삭제

# Get the index of 'Unnamed: 5'
start_idx = df.columns.get_loc('Unnamed: 5')

# Drop all columns from 'Unnamed: 5' to the end
df = df.iloc[:, :start_idx]


#6. 열(번수) 이름 변경
df.columns = ['번호', '성명', '학년', '세특']


# 6번 행을 삭제
df = df.drop(6)

# 인덱스 재설정 (선택사항)
df = df.reset_index(drop=True)


# remove rows with only NaN values
df = df.dropna(how='all')

# fill missing values in '이름' column with previous value
df['학년'] = df['학년'].fillna(method='ffill')

# Find the index of rows where the '번호' column is equal to '번 호'
rows_to_drop = df[df['번호'] == '번 호'].index

# Drop the rows by index
df = df.drop(rows_to_drop)

# fill missing values in '이름' column with previous value
df['번호'] = df['번호'].fillna(method='ffill')


# fill missing values in '이름' column with previous value
df['성명'] = df['성명'].fillna(method='ffill')


# Iterate through the DataFrame's index starting from the second row
df = df.drop(labels='학년', axis=1)

# '번호' 열을 기준으로 그룹화
grouped = df.groupby('성명')


# 각 그룹에서 '세특' 열의 문자열 값을 합치고, 나머지 열의 경우 첫 번째 값을 사용
def combine_special(row):
    return ', '.join(map(str, row))


result = grouped.agg({
    '번호': 'first',
    '세특': combine_special
})

# 인덱스 재설정 (선택사항)
result = result.reset_index()


#저장경로
path = '/content/drive/MyDrive/python/deungchon/result_s/'

#저장디렉토리
result.to_excel(f'{path}seteuk_306.xlsx', index=False, engine='openpyxl')

In [ ]:
#특정 열(변수)에 결측값이 있는지 확인
import pandas as pd

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to check
if df['세특'].isna().any():
    print("There are NA values in the column.")
else:
    print("No NA values found in the column.")

There are NA values in the column.


In [ ]:
#학년 열에서 '학 년'이면 앞학년으로 대치하는 코드: 아직 미완성~
import pandas as pd

# Read the Excel file into a pandas DataFrame
path = '/content/drive/MyDrive/python/deungchon/'
df = pd.read_csv(f'{path}srecord_306_raw_full.csv', encoding='cp949')


#1. 불필요한 행삭제: 1~3행 삭제
df = df.drop(range(0, 6))


#2. 이름과 세특 열 사이의 불필요한 'Unnamed: 3' column 삭제
df = df.drop(labels='Unnamed: 3', axis=1)


#3. remove rows with only NaN values
df = df.dropna(how='all')

#4. 비어있는 5번째열부터 끝까지 삭제

# Get the index of 'Unnamed: 5'
start_idx = df.columns.get_loc('Unnamed: 5')

# Drop all columns from 'Unnamed: 5' to the end
df = df.iloc[:, :start_idx]


#6. 열(번수) 이름 변경
df.columns = ['번호', '성명', '학년', '세특']


# 6번 행을 삭제
df = df.drop(6)

# 인덱스 재설정 (선택사항)
df = df.reset_index(drop=True)


# remove rows with only NaN values
df = df.dropna(how='all')

# fill missing values in '이름' column with previous value
df['학년'] = df['학년'].fillna(method='ffill')

# Find the index of rows where the '번호' column is equal to '번 호'
rows_to_drop = df[df['번호'] == '번 호'].index

# Drop the rows by index
df = df.drop(rows_to_drop)

# fill missing values in '이름' column with previous value
df['번호'] = df['번호'].fillna(method='ffill')


# fill missing values in '이름' column with previous value
df['성명'] = df['성명'].fillna(method='ffill')


# Iterate through the DataFrame's index starting from the second row
for i in df.index[1:]:
    # If the previous index doesn't exist, skip to the next iteration
    if i - 1 not in df.index:
        continue

    if df.loc[i, '학년'] == '학 년':
        df.loc[i, '학년'] = df.loc[i - 1, '학년']

#df.iloc[30:40, :]
df.head(20)